In [1]:
import os
import openai
from dotenv import find_dotenv, load_dotenv

# Load environment variables
_ = load_dotenv(find_dotenv())

# Set OpenAI API key
openai.api_type = os.getenv("api_type")
openai.api_base = os.getenv("api_base")
openai.api_version = os.getenv("api_version")
openai.api_key = os.getenv("OPENAI_API_KEY")

# RCI Chain with ChatModel

## Multi Chain

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import OpenAI

from langchain.schema.output_parser import StrOutputParser

In [3]:
model = AzureChatOpenAI(
        deployment_name="chatgpt-gpt35-turbo",
        model_name="gpt-35-turbo",
        temperature=0.7,
        max_tokens=1000
    )

In [4]:
prompt = ChatPromptTemplate.from_template(
    "tell me an intersting fact about {subject}"
    )

reverse_prompt = ChatPromptTemplate.from_template(
    "based on this interesting fact which is chunked down from a meta subject:\n\n {interesting_fact}\n\n Recover what the meta subject is\n Subject:"
    )

In [5]:
chain = prompt | model | StrOutputParser()

In [6]:
chain.invoke({"subject": "Laptop"})

'The first laptop computer, called the Grid Compass, was invented in 1982 and was priced at $8,000. It weighed 10 pounds and had a battery life of just 1 hour.'

In [7]:
import langchain

langchain.debug = True

In [8]:
chain1 = prompt | model | StrOutputParser()

chain2 = {"interesting_fact": chain1} | reverse_prompt | model | StrOutputParser()


In [9]:
chain2.invoke({"subject": "Siemens"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "Siemens"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": {
    "subject": "Siemens"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "Siemens"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "subject": "Siemens"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "chat",
    "ChatPromptValue"
  ],
  "kwargs": {
    "messages": [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain",
          "schema",
          "mess

[llm/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableSequence > 5:llm:AzureChatOpenAI] [2.72s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Siemens was the first company to develop a practical electric generator, which paved the way for the widespread use of electricity.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Siemens was the first company to develop a practical electric generator, which paved the way for the widespread use of electricity.",
            "additional_kwargs": {}
          }
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 24,
      "prompt_tokens": 17,
      "total_tokens": 41
    },
    "mod

'History of Electricity'

# Testing RCI

In [10]:
langchain.debug = False

In [11]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


In [12]:
template="You are a helpful assistant that imparts wisdom and guides people with accurate answers."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [13]:
chain1 = chat_prompt | model | StrOutputParser()

In [14]:
initial_question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"

In [15]:
initial_answer = chain1.invoke({"question": initial_question})
initial_answer

'Roger had 5 tennis balls and he bought 2 cans of tennis balls, each containing 3 tennis balls. So, he has a total of 5 + (2 x 3) = 11 tennis balls now.'

In [16]:
fake_initial_ai_answer = """Roger initially has 5 tennis balls. Each can of tennis balls contains 3 tennis balls, and he bought 2 cans, so he has 2 x 3 = 6 additional tennis balls.
Therefore, the total number of tennis balls Roger has now is 5 + 4 = 9."""

## Part 2 - Critique  

In [17]:
template="You are a helpful assistant that looks at answers and finds what is wrong with them based on the original question given."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="### Question:\n\n{question}\n\n ###Answer Given:{initial_answer}\n\n Review your previous answer and find problems with your answer"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [18]:
rc_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [19]:
chain2 = rc_prompt | model | StrOutputParser()

In [20]:
constructive_criticism = chain2.invoke({"question": initial_question, "initial_answer":fake_initial_ai_answer})
constructive_criticism

'The answer given has a mistake in the calculation. The answer should be 5 + 2 x 3 = 11. Roger initially has 5 tennis balls, and he buys 2 cans of tennis balls, which gives him a total of 2 x 3 = 6 additional tennis balls. Adding the initial 5 tennis balls to the 6 additional tennis balls gives a total of 11 tennis balls.'

## Part 3 - The Improvement

In [21]:
template="You are a helpful assistant that reviews answers and critiques based on the original question given and write a new improved final answer."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="### Question:\n\n{question}\n\n ###Answer Given:{initial_answer}\n\n \
###Constructive Criticism:{constructive_criticism}\n\n Based on the problems you found, improve your answer.\n\n### Final Answer:"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [22]:
improvement_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [23]:
chain3 = improvement_prompt | model | StrOutputParser()

In [24]:
final_result = chain3.invoke({"question": initial_question,
                              "initial_answer":fake_initial_ai_answer,
                              "constructive_criticism": constructive_criticism})

final_result

'Roger initially has 5 tennis balls. He buys 2 cans of tennis balls, which gives him a total of 2 x 3 = 6 additional tennis balls. Adding the initial 5 tennis balls to the 6 additional tennis balls gives a total of 11 tennis balls. Therefore, Roger has 11 tennis balls now.'

## Combined Chain

In [25]:
from operator import itemgetter

In [26]:

chain1 = chat_prompt | model | StrOutputParser()

critque_chain = {"question": itemgetter("question"),
                 "initial_answer": chain1 } | rc_prompt | model | StrOutputParser()

chain3 = {"question": itemgetter("question"),
          "initial_answer": chain1,
          "constructive_criticism": critque_chain} | improvement_prompt | model | StrOutputParser()

In [27]:
chain3.invoke({"question":"Write an sms message to say I am tired"})

'"I\'m tired."'

In [28]:
langchain.debug = True

In [29]:
chain3.invoke({"question":"Write an sms message to say I am tired"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap] Entering Chain run with input:
{
  "input": {
    "question": "Write an sms message to say I am tired"
  }
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableMap > 3:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": "Write an sms message to say I am tired"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 4:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "Write an sms message to say I am tired"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableMap > 4:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "quest

'"I\'m tired, need rest. Talk later."'